### Deliverable 3: Optimazing the model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Import dependencies for Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Import dependencies fro SVC model
from sklearn.svm import SVC

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
print(application_df.shape)
application_df.head()

(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


#### Column "SPECIAL_CONSIDERATIONS" was dropped in addition to columns "EIN" and "NAME"

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME','SPECIAL_CONSIDERATIONS'], axis = 1)
print(application_df.shape)
application_df.head()

(34299, 9)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
APPLICATION_TYPE = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(APPLICATION_TYPE[APPLICATION_TYPE < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
CLASSIFICATION = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(CLASSIFICATION[CLASSIFICATION < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

#### Column ASK_AMT was divided into bins in based of the amounts asked

In [6]:
# Look at ASK_AMT value counts for binning
ASK_AMT = application_df.ASK_AMT.value_counts()

# Determine which values to replace if counts are less than ..?
replace_amt = list(ASK_AMT[ASK_AMT < 5].index)

# Replace in dataframe
for amt in replace_amt:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt,"5K+")


# Convert ASK_AMT columns to str for encoding    
application_df['ASK_AMT'] = application_df['ASK_AMT'].astype(str)

# Check to make sure binning was successful
application_df.ASK_AMT.value_counts()

5000    25398
5K+      8901
Name: ASK_AMT, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE    9
AFFILIATION         6
CLASSIFICATION      6
USE_CASE            5
ORGANIZATION        4
INCOME_AMT          9
ASK_AMT             2
dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT_5000,ASK_AMT_5K+
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# Merge one-hot encoded features and drop the originals
#application_df = application_df.drop(['ASK_AMT'], axis=1)
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT_5000,ASK_AMT_5K+
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Attempt 1: Random Forest Classifier

In [11]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.728


### Attempt 2: SVM Model

In [12]:
# Create the SVM model
svm = SVC(kernel='linear')
# Train the model
svm.fit(X_train, y_train)
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.717


## Deep Neural with 3 hidden layers

### Attempt 3: Deep Neural with 3 hidden layers and 50 epochs (relu/sigmoid)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn_rs = tf.keras.models.Sequential()

# First hidden layer
nn_rs.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_rs.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_rs.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_rs.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_rs.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints3/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints3/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_rs.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5818 - accuracy: 0.7120
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5512 - accuracy: 0.7309
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5505 - accuracy: 0.7285
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7274
Epoch 5/50
781/804 [============================>.] - ETA: 0s - loss: 0.5487 - accuracy: 0.7341
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints3\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7341
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7370
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7325
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5516 - accuracy: 0.7264
Epoch 9/50
804/804 [============

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_rs.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 3 hidden layers and 50 epochs: {model_accuracy:.3f} (relu/sigmoid)")

268/268 - 0s - loss: 0.5591 - accuracy: 0.7261
Loss: 0.5590720772743225, Accuracy: 0.726064145565033
Deep Neural model accuracy with 3 hidden layers and 50 epochs: 0.726 (relu/sigmoid)


### Attempt 4: Deep Neural with 3 hidden layers and 50 epochs (sigmoid/relu)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn_sr = tf.keras.models.Sequential()

# First hidden layer
nn_sr.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn_sr.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn_sr.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_sr.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_sr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints4/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints4/weights.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_sr.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6185 - accuracy: 0.6768
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5719 - accuracy: 0.7271
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5698 - accuracy: 0.7240
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5656 - accuracy: 0.7237
Epoch 5/50
768/804 [===========================>..] - ETA: 0s - loss: 0.5600 - accuracy: 0.7292
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints4\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5600 - accuracy: 0.7291
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5596 - accuracy: 0.7277
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5583 - accuracy: 0.7320
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7331
Epoch 9/50
804/804 [============

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_sr.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 3 hidden layers and 50 epochs: {model_accuracy:.3f} (sigmoid/relu)")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7257
Loss: 0.5541034936904907, Accuracy: 0.7257142663002014
Deep Neural model accuracy with 3 hidden layers and 50 epochs: 0.726 (sigmoid/relu)


### Attempt 5: Deep Neural with 3 hidden layers and 50 epochs (sigmoid/sigmoid)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn_ss = tf.keras.models.Sequential()

# First hidden layer
nn_ss.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn_ss.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn_ss.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_ss.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_ss.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints5/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints5/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_ss.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6308 - accuracy: 0.6521
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5722 - accuracy: 0.7303
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5706 - accuracy: 0.7267
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5680 - accuracy: 0.7215
Epoch 5/50
772/804 [===========================>..] - ETA: 0s - loss: 0.5589 - accuracy: 0.7320
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints5\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5589 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7291
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7312
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5581 - accuracy: 0.7276
Epoch 9/50
804/804 [============

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_ss.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 3 hidden layers and 50 epochs: {model_accuracy:.3f} (sigmoid/sigmoid)")

268/268 - 0s - loss: 0.5531 - accuracy: 0.7298
Loss: 0.5530951619148254, Accuracy: 0.7297959327697754
Deep Neural model accuracy with 3 hidden layers and 50 epochs: 0.730 (sigmoid/sigmoid)


### Attempt 6: Deep Neural with 3 hidden layers and 50 epochs (relu/relu)

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn_rr = tf.keras.models.Sequential()

# First hidden layer
nn_rr.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_rr.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_rr.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_rr.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_rr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints6/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints6/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_rr.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.9781 - accuracy: 0.6752
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6093 - accuracy: 0.7202
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5933 - accuracy: 0.7222
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5954 - accuracy: 0.7167
Epoch 5/50
787/804 [============================>.] - ETA: 0s - loss: 0.5777 - accuracy: 0.7234
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints6\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5779 - accuracy: 0.7235
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5747 - accuracy: 0.7204
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5806 - accuracy: 0.7234
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5729 - accuracy: 0.7278
Epoch 9/50
804/804 [============

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_rr.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 3 hidden layers and 50 epochs: {model_accuracy:.3f} (relu/relu)")

268/268 - 0s - loss: 0.5753 - accuracy: 0.7236
Loss: 0.5752776861190796, Accuracy: 0.7236151695251465
Deep Neural model accuracy with 3 hidden layers and 50 epochs: 0.724 (relu/relu)


## Deep Neural with 2 hidden layers

### Attempt 7: Deep Neural with 2 hidden layers and 50 epochs (sigmoid/sigmoid)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_2ss = tf.keras.models.Sequential()

# First hidden layer
nn_2ss.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn_2ss.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn_2ss.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_2ss.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints7/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints7/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_2ss.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.6196 - accuracy: 0.6783
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5688 - accuracy: 0.7282
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5700 - accuracy: 0.7245
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5611 - accuracy: 0.7324
Epoch 5/50
799/804 [============================>.] - ETA: 0s - loss: 0.5649 - accuracy: 0.7252
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints7\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5649 - accuracy: 0.7252
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5590 - accuracy: 0.7278
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5514 - accuracy: 0.7328
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7283
Epoch 9/50
804/804 [============

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2ss.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 2 hidden layers and 50 epochs: {model_accuracy:.3f} (sigmoid/sigmoid)")

268/268 - 0s - loss: 0.5528 - accuracy: 0.7273
Loss: 0.5527995228767395, Accuracy: 0.7273469567298889
Deep Neural model accuracy with 2 hidden layers and 50 epochs: 0.727 (sigmoid/sigmoid)


### Attempt 8: Deep Neural with 2 hidden layers and 50 epochs (relu/relu)

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_2rr = tf.keras.models.Sequential()

# First hidden layer
nn_2rr.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_2rr.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn_2rr.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_2rr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints8/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints8/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_2rr.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 1.6718 - accuracy: 0.6612
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 1.3955 - accuracy: 0.6970
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 1.1936 - accuracy: 0.6899
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 1.3688 - accuracy: 0.7019
Epoch 5/50
761/804 [===========================>..] - ETA: 0s - loss: 1.2072 - accuracy: 0.7064
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints8\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 1.2010 - accuracy: 0.7062
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.8152 - accuracy: 0.7208
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.7382 - accuracy: 0.7232
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.7059 - accuracy: 0.7285
Epoch 9/50
804/804 [============

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2rr.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 2 hidden layers and 50 epochs: {model_accuracy:.3f} (relu/relu)")

268/268 - 0s - loss: 0.5825 - accuracy: 0.7236
Loss: 0.5824697613716125, Accuracy: 0.7236151695251465
Deep Neural model accuracy with 2 hidden layers and 50 epochs: 0.724 (relu/relu)


### Attempt 9: Deep Neural with 2 hidden layers and 50 epochs (sigmoid/relu)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_2sr = tf.keras.models.Sequential()

# First hidden layer
nn_2sr.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn_2sr.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn_2sr.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_2sr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints9/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints9/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_2sr.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 8.1790 - accuracy: 0.4698
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 8.2837 - accuracy: 0.4630
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 8.1898 - accuracy: 0.4691
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 8.2339 - accuracy: 0.4662
Epoch 5/50
793/804 [============================>.] - ETA: 0s - loss: 8.2502 - accuracy: 0.4651
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints9\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 8.2494 - accuracy: 0.4652
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 8.1912 - accuracy: 0.4690
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 8.1152 - accuracy: 0.4739
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 8.1551 - accuracy: 0.4713
Epoch 9/50
804/804 [============

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2sr.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 2 hidden layers and 50 epochs: {model_accuracy:.3f} (sigmoid/relu)")

268/268 - 0s - loss: 8.2800 - accuracy: 0.4632
Loss: 8.280006408691406, Accuracy: 0.4632070064544678
Deep Neural model accuracy with 2 hidden layers and 50 epochs: 0.463 (sigmoid/relu)


## Deep Neural with 1 hidden layer

### Attempt 10: Deep Neural with 1 hidden layers and 50 epochs (sigmoid/relu)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80

nn_1sr = tf.keras.models.Sequential()

# First hidden layer
nn_1sr.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)


# Output layer
nn_1sr.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Compile the Sequential model together and customize metrics
nn_1sr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints10/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints10/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_1sr.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 934us/step - loss: 1.3934 - accuracy: 0.6554
Epoch 2/50
804/804 [==============================] - 1s 959us/step - loss: 0.9737 - accuracy: 0.6581
Epoch 3/50
804/804 [==============================] - 1s 961us/step - loss: 0.6131 - accuracy: 0.7099
Epoch 4/50
804/804 [==============================] - 1s 929us/step - loss: 0.6069 - accuracy: 0.7206
Epoch 5/50
765/804 [===========================>..] - ETA: 0s - loss: 6.0956 - accuracy: 0.5216
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints10\weights.05.hdf5
804/804 [==============================] - 1s 954us/step - loss: 6.1138 - accuracy: 0.5206
Epoch 6/50
804/804 [==============================] - 1s 948us/step - loss: 5.8987 - accuracy: 0.5143
Epoch 7/50
804/804 [==============================] - 1s 935us/step - loss: 5.8477 - accuracy: 0.5272
Epoch 8/50
804/804 [==============================] - 1s 934us/step - loss: 5.4397 - accuracy: 0.5391
Epoch 9/50
804/

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1sr.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 1 hidden layer and 50 epochs: {model_accuracy:.3f} (sigmoid/relu)")

268/268 - 0s - loss: 0.5948 - accuracy: 0.7185
Loss: 0.594789445400238, Accuracy: 0.7184839844703674
Deep Neural model accuracy with 1 hidden layer and 50 epochs: 0.718 (sigmoid/relu)


### Attempt 11: Deep Neural with 1 hidden layer and 50 epochs (relu/sigmoid)

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80

nn_1rs = tf.keras.models.Sequential()

# First hidden layer
nn_1rs.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Output layer
nn_1rs.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_1rs.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define the checkpoint path and filenames
os.makedirs("Optimization_Checkpoints/checkpoints11/",exist_ok=True)
checkpoint_path = "Optimization_Checkpoints/checkpoints11/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

# Train the model
fit_model = nn_1rs.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 970us/step - loss: 0.6091 - accuracy: 0.6785
Epoch 2/50
804/804 [==============================] - 1s 948us/step - loss: 0.5537 - accuracy: 0.7325
Epoch 3/50
804/804 [==============================] - 1s 921us/step - loss: 0.5528 - accuracy: 0.7309
Epoch 4/50
804/804 [==============================] - 1s 964us/step - loss: 0.5457 - accuracy: 0.7354
Epoch 5/50
772/804 [===========================>..] - ETA: 0s - loss: 0.5526 - accuracy: 0.7283
Epoch 00005: saving model to Optimization_Checkpoints/checkpoints11\weights.05.hdf5
804/804 [==============================] - 1s 996us/step - loss: 0.5525 - accuracy: 0.7284
Epoch 6/50
804/804 [==============================] - 1s 982us/step - loss: 0.5498 - accuracy: 0.7295
Epoch 7/50
804/804 [==============================] - 1s 953us/step - loss: 0.5427 - accuracy: 0.7352
Epoch 8/50
804/804 [==============================] - 1s 963us/step - loss: 0.5508 - accuracy: 0.7297
Epoch 9/50
804/

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1rs.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural model accuracy with 1 hidden layer and 50 epochs: {model_accuracy:.3f} (relu/sigmoid)")

268/268 - 0s - loss: 0.5578 - accuracy: 0.7271
Loss: 0.5578204393386841, Accuracy: 0.7271137237548828
Deep Neural model accuracy with 1 hidden layer and 50 epochs: 0.727 (relu/sigmoid)


In [31]:
# Export all the models to HDF5 files
os.makedirs("AlphabetSoupCharity_Optimization/",exist_ok=True)
nn_rs.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_3Atempt.h5")
nn_sr.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_4Atempt.h5")
nn_ss.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_5Atempt.h5")
nn_rr.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_6Atempt.h5")

nn_2ss.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_7Atempt.h5")
nn_2rr.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_8Atempt.h5")
nn_2sr.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_9Atempt.h5")

nn_1sr.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_10Atempt.h5")
nn_1rs.save("AlphabetSoupCharity_Optimization/AlphabetSoupCharity_Optimization_11Atempt.h5")